In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import operator

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)

In [3]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, mean, std_dev):
    return (x-mean)/std_dev
    

In [4]:
x_train, y_train, x_valid, y_valid = get_data()


#### normalize the train & valida input data.
see we use training values for valid data normalization

In [5]:
x_train_mean, x_train_std = x_train.mean(), x_train.std()
x_train_mean, x_train_std 

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, x_train_mean, x_train_std)
x_valid = normalize(x_valid, x_train_mean, x_train_std)

#### see diff in mean & std values

In [7]:
x_train_mean, x_train_std = x_train.mean(), x_train.std()
x_train_mean, x_train_std 

(tensor(3.0614e-05), tensor(1.))

In [8]:
def test_near_zero(a, tol=1e-3):
    assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [12]:
n, m = x_train.shape  
c = y_train.max()  + 1 # 1 extra class for the input which we cannot classify
n, m, c

(50000, 784, tensor(10))

### ARCHITECTURE

#### linear, relu, kaiming init & model

 we will first see the effect of random initialization vs [Kaiming initialization 2.2](https://arxiv.org/abs/1502.01852)

In [14]:
nh = 50 # hidden layers
# randn: Returns a tensor filled with random numbers from a normal distribution with mean `0` and variance `1`
w1 = torch.randn(m, nh)/math.sqrt(m)
w2 = torch.randn(nh, 1)/math.sqrt(m)
b1 = torch.zeros(nh)
b2 = torch.zeros(1)


In [19]:
test_near_zero(w1.mean())
test_near_zero(w1.std()- 1/math.sqrt(m))

In [20]:
x_valid.mean(), x_valid.std() # close to 0 & 1

(tensor(-0.0058), tensor(0.9924))

In [23]:
def lin(x, w, b):
    # linear layer
    return x@w + b

t = lin(x_valid, w1, b1)
t.mean(), t.std()

(tensor(0.0274), tensor(1.0092))

In [24]:
def relu(x):
    return x.clamp_min(0.)

t1 = relu(t) # t from lin(..)
t1.mean(), t1.std()

(tensor(0.4094), tensor(0.6151))

In [42]:
def kaiming_init(m, nh):
    return torch.randn(m, nh) * math.sqrt(2./m)

In [46]:
w1 = kaiming_init(m , nh)
t = relu(lin(x_valid, w1, b1))
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(-0.0004), tensor(0.0503), tensor(0.0290), tensor(0.7870))

##### we can compare this kaiming_init with Pytorch kaiminig init

In [27]:
from torch.nn import init

In [32]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out") 
#Choosing `fan_in` preserves the magnitude of the variance of the weights in the
#forward pass. Choosing `fan_out` preserves the magnitudes in the backwards pass.
t = relu(lin(x_valid, w1, b1))

In [33]:
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(0.0002), tensor(0.0505), tensor(0.5844), tensor(0.8174))

#### our relu is having mean > 0.5. Let's clip at 0.5 mercilessly.

In [44]:
def relu(x):
    return x.clamp_min(0.) - 0.5

In [45]:
w1 = kaiming_init(m, nh)
t1 = relu(lin(x_valid, w1, b1))
w1.mean(), w1.std(), t.mean(), t.std()

(tensor(0.0002), tensor(0.0504), tensor(0.0133), tensor(0.7848))

### Our First model fun

In [47]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [48]:
w2 = kaiming_init(nh, 1)

In [49]:
%timeit -n 10 _= model(x_valid)

8.71 ms ± 98 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

### Our First Loss Fun Mean Sq Error (MSE). 
to keep this simple

In [74]:
# squeeze will remove ,1] ; unsqueeze does the opposite
model(x_valid).shape, model(x_valid).squeeze(-1).shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [75]:
def mse(output, targ):
    return (output.squeeze(-1) - targ).pow(2).mean()

In [76]:
y_train, y_valid = y_train.float(), y_valid.float()

#### predict the output

In [81]:
preds = model(x_train)
x_train.shape, preds.shape, y_train.shape
# 50000 samples of 784 col (28 * 28 pic)

(torch.Size([50000, 784]), torch.Size([50000, 1]), torch.Size([50000]))

In [82]:
mse(preds, y_train)

tensor(29.0187)